# Run BUILD baseline model on custom input text.
## Running following cells will create User Interface where you can input the judgement text and see predicted rhetorical roles & AI generated summary for each rhetorical role. Please click on "Runtime" on top bar and click "Run all" .
### The next cell performs the setup & takes 5 minutes to run. After completion , it will automatically try to open UI in seperate tab but may get blocked due to popup blocking. If it does not then click on the gradio link that you will see at the end of the cell output. You can use the default judgement text or copy paste custom judgement text (E.g.https://indiankanoon.org/doc/181002261/)  into input box and click submit. If you do not change the text in input box then results will be shown immediately. Else you will have to wait 3 to 15 minutes for results depending on the length of input text. This app works best for Criminal & Tax cases. 

In [ ]:
!pip install gradio==2.6.0
import gradio as gr
import requests
import json

!pip install --upgrade spacy
!pip install allennlp
!pip install --upgrade google-cloud-storage
!pip install --upgrade google-api-core
!pip install --upgrade google-auth
!pip install --upgrade google-cloud-core

!git clone https://github.com/Legal-NLP-EkStep/rhetorical-role-baseline.git
#!pip install -r rhetorical-role-baseline/requirements_colab.txt
!python -m spacy download en_core_web_trf
!pip install --upgrade huggingface-hub

%cd rhetorical-role-baseline
######## download model file
model_url = 'https://storage.googleapis.com/indianlegalbert/OPEN_SOURCED_FILES/Rhetorical_Role_Benchmark/Model/model.pt'
model = requests.get(model_url).content
with open("datasets/model.pt", "wb") as file:
    file.write(model)

from data_prep import split_preamble_judgement
from SummaryGeneration import SummaryGeneration
def concatenate_text_by_rhetorical_role(annotation_list):
    #### concatenate text for each rhetorical role
    rhetorical_rolewise_text = {} # keys are rhetorical roles and value is concatenated text
    for annotation in annotation_list:
      sent_role = annotation['value']['labels'][0]
      sent_txt = annotation['value']['text'].strip()
      if rhetorical_rolewise_text.get(sent_role) is None:
        rhetorical_rolewise_text[sent_role] = sent_txt
      else:
        if not rhetorical_rolewise_text[sent_role].endswith('.'):
          rhetorical_rolewise_text[sent_role] = rhetorical_rolewise_text[sent_role] + '. '+ sent_txt
        else:
          rhetorical_rolewise_text[sent_role] = rhetorical_rolewise_text[sent_role] + ' '+ sent_txt
    return rhetorical_rolewise_text

def predict_rhetorical_roles_and_create_summary(input_txt):

    if input_txt == '''                                                         REPORTABLE\n\n  IN THE SUPREME COURT OF INDIA\n CRIMINAL APPELLATE JURISDICTION\n\nCRIMINAL APPEAL NO. 1053 OF 2016\n\nNARESH KUMAR alias NITU                            ..........APPELLANT\n                                                   ......RESPONDENT\n                                           Versus\nSTATE OF HIMACHAL PRADESH\n\n                                       JUDGMENT\n NAVIN SINHA, J.\n The acquittal of the appellant by the Special Judge, Shimla in Sessions Trial No.7-S/7/2012, from the charge under Sections 20 and 61 of the Narcotic Drugs and Psychotropic Substances Act, 1985 (hereinafter referred as `the Act') has been reversed by the High Court. The appellant has been sentenced to fifteen years imprisonment and fine of Rs.2,00,000/-. Thus the present appeal.\n 2. The appellant is stated to have been apprehended at Majhotli, by the police party, on suspicion, at about 6.15 A.M.\nafter he disembarked from the State public transport bus going from Nerwa to Chamunda. Two kilograms of Charas is stated to have been recovered from a bag in his possession in presence of PW-2, Sita Ram an independent witness. DW-1 Shayam Singh, the depot in-charge at Nerwa, and DW-2 Khem Raj, the conductor of the bus in question were examined as defence witnesses.\n 3. After consideration of the entirety of the evidence, particularly that of PW-2 and DW-2, and also noticing that PW-1, Constable Rakesh Kumar, an eye-witness to the incident, had not been mentioned as a witness in the seizure memo Exhibit PW-1/B, the Special Judge opined that two theories had emerged with regard to the accusations against the appellant. The appellant was acquitted, giving him the benefit of doubt.\n 4. The High Court reversed the acquittal, holding that PW-2 had admitted his signatures on Exhibit PW-1/B, the bag along with the narcotic, Ex.PW-2/A seal impression, Ex.PW-2/D the arrest memo and the Ex.PW-2/E personal search memo. No complaint had been lodged by the witness that he had been compelled by the police to sign the documents under pressure.\nThe statements of the official witnesses, PW-1 Rakesh Kumar and PW-6 Head Constable Parmanand, were trustworthy, inspiring confidence, and could not be rejected only on the ground that they were police personnel. Any discrepancy with regard to distance and travelling time between Nerwa and Majhotli could be attributed to memory loss with passage of time, and was not required to be with mathematical precision. The time with regard to purchase of bus ticket had not been established.\n 5. Learned counsel for the appellant submits that he has been falsely implicated. He was deboarded from the bus at Chopal after it arrived from Majhotli and the narcotics planted on him. There had been no recovery from him. The bus ticket evidently reveals that he boarded the bus at 6:51 A.M. The question of his being apprehended at 6:15 A.M. simply does not arise. The road distance from Nerwa to Majhotli was 26 kms. as deposed by DW-1. In the hills, it would have taken the bus at least one hour or more, to cover this distance. The bus could not have reached Majhotli till 8:00 A.M. or thereafter. PW-2, the independent witness, deposed that he was stopped by the police at about 10.30 A.M. at Majhotli and after checking his motor cycle papers was allowed to leave. He was called to the Police Station at 1.00 P.M. and asked to put his signatures on certain documents. The witness denied any search, seizure and recovery from the appellant in his presence. If an independent witness was available, and relied upon by the prosecution, his evidence could not be discarded without reason, to hold that the police version was the truth. Additional submissions were made with regard to non-compliance with Section 50 of the Act, as there was personal search also, and that the complainant himself could not be the investigating officer. Reliance was placed on Basappa vs. State of Karnataka, 2014 (5) SCC 154 that if two views were possible, the acquittal ought not to have been interfered with in appeal.\n 6. We have considered the submissions on behalf of the parties, and also examined the evidence and other materials on record.\n 7. The public bus, on which the appellant was traveling, was going from Nerwa to Chamunda. The ticket issued to the appellant Exhibit DX, proved by the bus Conductor DW-2, bears the time of issuance 6.51 A.M., visible to the naked eye. The distance from Nerwa to Majhotli, is 26 kms. as deposed by DW-1.\nWe find substance in the submission on behalf of the appellant, that the travelling time for the bus, in the hills, for this distance would be one hour or more. Prima facie, the prosecution story that the appellant was apprehended at Majhotli at 6.15 A.M.\nbecomes seriously doubtful if not impossible. The bus would have reached Majhotli at about 8.00 A.M. or thereafter only. The conclusion of the High Court that passage of time, and memory loss, were sufficient explanation for the time difference, is held to be perverse, and without proper consideration of Exhibit DX.\nPW-2, the independent witness has stated that he was stopped at Majhotli by the police at 10.30 A.M. and was allowed to leave after verification of his motor cycle papers. The witness has specifically denied that the appellant was apprehended in his presence and that any search, seizure and recovery was conducted in his presence. He had deposed that he was called to the police station at 1:00 P.M. and asked to sign the papers. The witness was declared hostile. This aspect has not been considered by the High Court, which proceeded on the only assumption that the signatures were admitted.\n 8. In a case of sudden recovery, independent witness may not be available. But if an independent witness is available, and the prosecution initially seeks to rely upon him, it cannot suddenly discard the witness because it finds him inconvenient, and place reliance upon police witnesses only. In the stringent nature of the provisions of the Act, the reverse burden of proof, the presumption of culpability under Section 35, and the presumption against the accused under Section 54, any reliance upon Section 114 of the Evidence Act in the facts of the present case, can only be at the risk of a fair trial to the accused.\nKaramjit Singh vs. State (Delhi Administration), AIR 2003 SC 1311, is distinguishable on its facts as independent witness had refused to sign because of the fear of terrorists. Likewise S.\n Jeevananthanan vs. State, 2004(5) SCC 230, also does not appear to be a case where independent witnesses were available.\n 9. The presumption against the accused of culpability under Section 35, and under Section 54 of the Act to explain possession satisfactorily, are rebuttable. It does not dispense with the obligation of the prosecution to prove the charge beyond all reasonable doubt. The presumptive provision with reverse burden of proof, does not sanction conviction on basis of preponderance of probability. Section 35 (2) provides that a fact can be said to have been proved if it is established beyond reasonable doubt and not on preponderance of probability. That the right of the accused to a fair trial could not be whittled down under the Act was considered in Noor Aga vs. State of Punjab, (2008) 16 SCC 417, observing:- \"58......An initial burden exists upon the prosecution and only when it stands satisfied, would the legal burden shift. Even then, the standard of proof required for the accused to prove his innocence is not as high as that of the prosecution. Whereas the standard of proof required to prove the guilt of the accused on the prosecution is \"beyond all reasonable doubt\" but it is \"preponderance of probability\" on the accused. If the prosecution fails to prove the foundational facts so as to attract the rigours of Section 35 of the Act, the actus reus which is possession of contraband by the accused cannot be said to have been established.\n 59. With a view to bring within its purview the requirements of Section 54 of the Act, element of possession of the contraband was essential so as to shift the burden on the accused. The provisions being exceptions to the general rule, the generality thereof would continue to be operative, namely, the element of possession will have to be proved beyond reasonable doubt.\"\n 10. In the facts of the present case, and the nature of evidence as discussed, the prosecution had failed to establish the foundational facts beyond all reasonable doubt. The special judge committed no error in acquitting the appellant. The High Court ought not to have interfered with the same. The submissions regarding non-compliance with Section 50 of the Act, or that the complainant could not be the investigating officer are not considered necessary to deal with in the facts of the case.\n 11. In Basappa (supra), it was observed that the High Court before setting aside an order of acquittal was required to record a finding that the conclusions of the Trial Court were so perverse and wholly unreasonable, so as not to be a plausible view by misreading and incorrect appreciation of evidence. The conclusions of the High Court in the facts of the present case are more speculative, based on conjectures and surmises, contrary to the weight of the evidence on record.\n 12. The order of the High Court is set aside. The acquittal of the appellant ordered by the Special Judge is restored. The appellant is set at liberty forthwith, unless wanted in any other case. The appeal is allowed.\n ........................................J.\n                             (L. Nageswara Rao) .......................................J.\n                                     (Navin Sinha) New Delhi, July 27, 2017''':
      predefined_summary = requests.get('https://storage.googleapis.com/indianlegalbert/Rhetorical%20role/experiments/predefined_summary.txt').text
      predefined_rhet_roles =  requests.get('https://storage.googleapis.com/indianlegalbert/Rhetorical%20role/experiments/predefined_rhet.json').json()
      return predefined_summary,predefined_rhet_roles

    preamble_txt,preamble_removed_txt  = split_preamble_judgement(input_txt)
    cleaned_text = preamble_txt + preamble_removed_txt
    custom_data = [  {"id": 1,"data": { "text": cleaned_text }}]
    with open('datasets/custom_data.json','w') as file:
      json.dump(custom_data,file)
    
    !python infer_data_prep.py datasets/custom_data.json datasets/custom_processed_data.json
    !python infer_new.py datasets/custom_processed_data.json datasets/custom_output.json datasets/model.pt

    predictions = json.load(open('datasets/custom_output.json'))
    sent_tags = []
    annotation_list = predictions[0]['annotations'][0]['result']
    for i,annotation in enumerate(annotation_list):
      sent_tags.append((annotation['value']['text'],annotation['value']['labels'][0]))
      if i < len(annotation_list)-1 and annotation_list[i+1]['value']['start'] > annotation['value']['end']:
        sent_tags.append((cleaned_text[annotation['value']['end'] :annotation_list[i+1]['value']['start']],None))

    ##### create summary for each rhetorical role
    rheotrical_roles_to_summarize = ['FAC','RLC','ISSUE','ARG_PETITIONER','ARG_RESPONDENT','ANALYSIS','PRECEDENT_RELIED','PRECEDENT_NOT_RELIED','STA','RATIO','RPC']
    
    try:
      rhetorical_rolewise_text = concatenate_text_by_rhetorical_role(annotation_list)
      legal_summarizer = SummaryGeneration(model="nsi319/legal-pegasus", tokenizer="nsi319/legal-pegasus")
      
      generated_summaries_dict={} ##### key is rhetorical role and value is dict of {"summary":'fdsf...', "is_summarized":True}
      for rhet_role, rhet_text in rhetorical_rolewise_text.items():
        ######## create summary only if the text is long enough
        if len(rhet_text)>1000:
          generated_summaries = legal_summarizer.generate(rhet_text, token_max_length=1024)
          generated_summaries_dict[rhet_role] = {"summary":generated_summaries[0]["level_1_summary"],"is_summarized":True}
        else:
          generated_summaries_dict[rhet_role] = {"summary":rhet_text,"is_summarized":False}
      final_summary=''
      for rhetorical_role in rheotrical_roles_to_summarize:
        if generated_summaries_dict.get(rhetorical_role) is not None:
          final_summary = final_summary + '#'*30+'\n' +rhetorical_role +" SUMMARY:\n" + generated_summaries_dict[rhetorical_role]['summary']+ "\n"+ '#'*30+'\n'
    except Exception as e:
      final_summary = "ERROR in creating summary, please try again.\n Error Description:" + str(e)

    return final_summary,sent_tags

input_txt = '''                                                         REPORTABLE\n\n  IN THE SUPREME COURT OF INDIA\n CRIMINAL APPELLATE JURISDICTION\n\nCRIMINAL APPEAL NO. 1053 OF 2016\n\nNARESH KUMAR alias NITU                            ..........APPELLANT\n                                                   ......RESPONDENT\n                                           Versus\nSTATE OF HIMACHAL PRADESH\n\n                                       JUDGMENT\n NAVIN SINHA, J.\n The acquittal of the appellant by the Special Judge, Shimla in Sessions Trial No.7-S/7/2012, from the charge under Sections 20 and 61 of the Narcotic Drugs and Psychotropic Substances Act, 1985 (hereinafter referred as `the Act') has been reversed by the High Court. The appellant has been sentenced to fifteen years imprisonment and fine of Rs.2,00,000/-. Thus the present appeal.\n 2. The appellant is stated to have been apprehended at Majhotli, by the police party, on suspicion, at about 6.15 A.M.\nafter he disembarked from the State public transport bus going from Nerwa to Chamunda. Two kilograms of Charas is stated to have been recovered from a bag in his possession in presence of PW-2, Sita Ram an independent witness. DW-1 Shayam Singh, the depot in-charge at Nerwa, and DW-2 Khem Raj, the conductor of the bus in question were examined as defence witnesses.\n 3. After consideration of the entirety of the evidence, particularly that of PW-2 and DW-2, and also noticing that PW-1, Constable Rakesh Kumar, an eye-witness to the incident, had not been mentioned as a witness in the seizure memo Exhibit PW-1/B, the Special Judge opined that two theories had emerged with regard to the accusations against the appellant. The appellant was acquitted, giving him the benefit of doubt.\n 4. The High Court reversed the acquittal, holding that PW-2 had admitted his signatures on Exhibit PW-1/B, the bag along with the narcotic, Ex.PW-2/A seal impression, Ex.PW-2/D the arrest memo and the Ex.PW-2/E personal search memo. No complaint had been lodged by the witness that he had been compelled by the police to sign the documents under pressure.\nThe statements of the official witnesses, PW-1 Rakesh Kumar and PW-6 Head Constable Parmanand, were trustworthy, inspiring confidence, and could not be rejected only on the ground that they were police personnel. Any discrepancy with regard to distance and travelling time between Nerwa and Majhotli could be attributed to memory loss with passage of time, and was not required to be with mathematical precision. The time with regard to purchase of bus ticket had not been established.\n 5. Learned counsel for the appellant submits that he has been falsely implicated. He was deboarded from the bus at Chopal after it arrived from Majhotli and the narcotics planted on him. There had been no recovery from him. The bus ticket evidently reveals that he boarded the bus at 6:51 A.M. The question of his being apprehended at 6:15 A.M. simply does not arise. The road distance from Nerwa to Majhotli was 26 kms. as deposed by DW-1. In the hills, it would have taken the bus at least one hour or more, to cover this distance. The bus could not have reached Majhotli till 8:00 A.M. or thereafter. PW-2, the independent witness, deposed that he was stopped by the police at about 10.30 A.M. at Majhotli and after checking his motor cycle papers was allowed to leave. He was called to the Police Station at 1.00 P.M. and asked to put his signatures on certain documents. The witness denied any search, seizure and recovery from the appellant in his presence. If an independent witness was available, and relied upon by the prosecution, his evidence could not be discarded without reason, to hold that the police version was the truth. Additional submissions were made with regard to non-compliance with Section 50 of the Act, as there was personal search also, and that the complainant himself could not be the investigating officer. Reliance was placed on Basappa vs. State of Karnataka, 2014 (5) SCC 154 that if two views were possible, the acquittal ought not to have been interfered with in appeal.\n 6. We have considered the submissions on behalf of the parties, and also examined the evidence and other materials on record.\n 7. The public bus, on which the appellant was traveling, was going from Nerwa to Chamunda. The ticket issued to the appellant Exhibit DX, proved by the bus Conductor DW-2, bears the time of issuance 6.51 A.M., visible to the naked eye. The distance from Nerwa to Majhotli, is 26 kms. as deposed by DW-1.\nWe find substance in the submission on behalf of the appellant, that the travelling time for the bus, in the hills, for this distance would be one hour or more. Prima facie, the prosecution story that the appellant was apprehended at Majhotli at 6.15 A.M.\nbecomes seriously doubtful if not impossible. The bus would have reached Majhotli at about 8.00 A.M. or thereafter only. The conclusion of the High Court that passage of time, and memory loss, were sufficient explanation for the time difference, is held to be perverse, and without proper consideration of Exhibit DX.\nPW-2, the independent witness has stated that he was stopped at Majhotli by the police at 10.30 A.M. and was allowed to leave after verification of his motor cycle papers. The witness has specifically denied that the appellant was apprehended in his presence and that any search, seizure and recovery was conducted in his presence. He had deposed that he was called to the police station at 1:00 P.M. and asked to sign the papers. The witness was declared hostile. This aspect has not been considered by the High Court, which proceeded on the only assumption that the signatures were admitted.\n 8. In a case of sudden recovery, independent witness may not be available. But if an independent witness is available, and the prosecution initially seeks to rely upon him, it cannot suddenly discard the witness because it finds him inconvenient, and place reliance upon police witnesses only. In the stringent nature of the provisions of the Act, the reverse burden of proof, the presumption of culpability under Section 35, and the presumption against the accused under Section 54, any reliance upon Section 114 of the Evidence Act in the facts of the present case, can only be at the risk of a fair trial to the accused.\nKaramjit Singh vs. State (Delhi Administration), AIR 2003 SC 1311, is distinguishable on its facts as independent witness had refused to sign because of the fear of terrorists. Likewise S.\n Jeevananthanan vs. State, 2004(5) SCC 230, also does not appear to be a case where independent witnesses were available.\n 9. The presumption against the accused of culpability under Section 35, and under Section 54 of the Act to explain possession satisfactorily, are rebuttable. It does not dispense with the obligation of the prosecution to prove the charge beyond all reasonable doubt. The presumptive provision with reverse burden of proof, does not sanction conviction on basis of preponderance of probability. Section 35 (2) provides that a fact can be said to have been proved if it is established beyond reasonable doubt and not on preponderance of probability. That the right of the accused to a fair trial could not be whittled down under the Act was considered in Noor Aga vs. State of Punjab, (2008) 16 SCC 417, observing:- \"58......An initial burden exists upon the prosecution and only when it stands satisfied, would the legal burden shift. Even then, the standard of proof required for the accused to prove his innocence is not as high as that of the prosecution. Whereas the standard of proof required to prove the guilt of the accused on the prosecution is \"beyond all reasonable doubt\" but it is \"preponderance of probability\" on the accused. If the prosecution fails to prove the foundational facts so as to attract the rigours of Section 35 of the Act, the actus reus which is possession of contraband by the accused cannot be said to have been established.\n 59. With a view to bring within its purview the requirements of Section 54 of the Act, element of possession of the contraband was essential so as to shift the burden on the accused. The provisions being exceptions to the general rule, the generality thereof would continue to be operative, namely, the element of possession will have to be proved beyond reasonable doubt.\"\n 10. In the facts of the present case, and the nature of evidence as discussed, the prosecution had failed to establish the foundational facts beyond all reasonable doubt. The special judge committed no error in acquitting the appellant. The High Court ought not to have interfered with the same. The submissions regarding non-compliance with Section 50 of the Act, or that the complainant could not be the investigating officer are not considered necessary to deal with in the facts of the case.\n 11. In Basappa (supra), it was observed that the High Court before setting aside an order of acquittal was required to record a finding that the conclusions of the Trial Court were so perverse and wholly unreasonable, so as not to be a plausible view by misreading and incorrect appreciation of evidence. The conclusions of the High Court in the facts of the present case are more speculative, based on conjectures and surmises, contrary to the weight of the evidence on record.\n 12. The order of the High Court is set aside. The acquittal of the appellant ordered by the Special Judge is restored. The appellant is set at liberty forthwith, unless wanted in any other case. The appeal is allowed.\n ........................................J.\n                             (L. Nageswara Rao) .......................................J.\n                                     (Navin Sinha) New Delhi, July 27, 2017'''

color_map={"PREAMBLE": "LightCyan", "FAC": "Yellow","RLC":"LightSkyBlue",
"ISSUE":"Aquamarine","ARG_PETITIONER":"LightGreen","ARG_RESPONDENT":"Lavender","ANALYSIS":"Thistle","STA":"PaleGoldenrod","PRE_RELIED":"YellowGreen","PRE_NOT_RELIED":"Gold",
"RATIO":"Orange","RPC":"PeachPuff","NONE":"Linen" }


########## launch Gradio UI
iface = gr.Interface(
    predict_rhetorical_roles_and_create_summary,
    [
        gr.inputs.Textbox(
            lines=20, default=input_txt)
    ],
    [gr.outputs.Textbox(label = 'AI Generated Summary'), gr.outputs.HighlightedText(color_map = color_map, label = 'Predicted Rhetorical Roles')],
    description = "You can copy paste custom judgement text (E.g. https://indiankanoon.org/doc/181002261/) into input box and click submit. If you do not change the text in input box then results will be shown immediately. Else you will have to wait (3 to 15 minutes depending on length of input text) for results",
    enable_queue=True)
gradio_output = iface.launch()

########## play beep 
from google.colab import output
output.eval_js('new Audio("https://static.sfdict.com/audio/C07/C0702600.mp3").play()')

## launch UI in new tab
js_script= "window.open('"+ gradio_output[2]+ "'"+',"_blank");'
output.eval_js(js_script)

     |████████████████████████████████| 938 kB 5.3 MB/s 
     |████████████████████████████████| 2.0 MB 37.7 MB/s 
     |████████████████████████████████| 211 kB 45.1 MB/s 
     |████████████████████████████████| 3.6 MB 11.9 MB/s 
     |████████████████████████████████| 856 kB 8.4 MB/s 
     |████████████████████████████████| 61 kB 273 kB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4712 sha256=577f4a4736a37773959deca890f7e0b2dc82f62d7580e445b2c7d714e9d88430
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3373 sha256=2f8be491c3ccb032d3d4463b0ef04a377e0f5adcd8508b3d3a6a1dc6ce402624
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster
     |████████████████████████████████| 6.0 MB 5.3 MB/s 
     |██████████████████

     |████████████████████████████████| 107 kB 5.1 MB/s 
     |████████████████████████████████| 114 kB 32.4 MB/s 
     |████████████████████████████████| 76 kB 5.3 MB/s 
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 1.26.3
    Uninstalling google-api-core-1.26.3:
      Successfully uninstalled google-api-core-1.26.3
  Attempting uninstall: google-resumable-media
    Found existing installation: google-resumable-media 0.4.1
    Uninstalling google-resumable-media-0.4.1:
      Successfully uninstalled google-resumable-media-0.4.1
  Attempting uninstall: google-cloud-core
    Found existing installation: google-cloud-core 1.0.3
    Uninstalling google-cloud-core-1.0.3:
      Successfully uninstalled google-cloud-core-1.0.3
  Attempting uninstall: google-cloud-storage
    Found existing installation: google-cloud-storage 1.18.1
    Uninstalling google-cloud-storage-1.18.1:
      Successfully uninstalled google-cloud-storage-1.18.1
ERROR: pip's de

     |████████████████████████████████| 156 kB 5.2 MB/s 
  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.35.0
    Uninstalling google-auth-1.35.0:
      Successfully uninstalled google-auth-1.35.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 1.21.0 requires google-cloud-core<2.0dev,>=1.0.3, but you have google-cloud-core 2.2.3 which is incompatible.
google-cloud-bigquery 1.21.0 requires google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you have google-resumable-media 2.3.2 which is incompatible.
firebase-admin 4.4.0 requires google-api-core[grpc]<2.0.0dev,>=1.14.0; platform_python_implementation != "PyPy", but you have google-api-core 2.7.1 which is incompatible.


Cloning into 'rhetorical-role-baseline'...
remote: Enumerating objects: 419, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 419 (delta 99), reused 170 (delta 63), pack-reused 203
Receiving objects: 100% (419/419), 861.18 KiB | 6.89 MiB/s, done.
Resolving deltas: 100% (198/198), done.
     |████████████████████████████████| 460.2 MB 31 kB/s 
     |████████████████████████████████| 51 kB 132 kB/s 
     |████████████████████████████████| 1.1 MB 11.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
/content/rhetorical-role-baseline


/usr/local/lib/python3.7/dist-packages/gradio/interface.py:200: UserWarning: The `enable_queue` parameter in the `Interface` will be deprecated. Please use the `enable_queue` parameter in `launch()` instead
  warnings.warn("The `enable_queue` parameter in the `Interface` will be deprecated. Please use the `enable_queue` parameter in `launch()` instead")


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://44503.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


MessageError: ignored

In [ ]:

import spacy
spacy.require_gpu()
nlp=spacy.load('en_core_web_trf')

In [ ]:
!pip freeze

In [ ]:
#@title
# ####### function to predict rhetorical roles
# from data_prep import split_preamble_judgement
# def predict_rhetorical_roles(input_txt):
#     preamble_txt,preamble_removed_txt  = split_preamble_judgement(input_txt)
#     cleaned_text = preamble_txt + preamble_removed_txt
#     custom_data = [  {"id": 1,"data": { "text": cleaned_text }}]
#     with open('datasets/custom_data.json','w') as file:
#       json.dump(custom_data,file)
    
#     !python infer_data_prep.py datasets/custom_data.json datasets/custom_processed_data.json
#     !python infer_new.py datasets/custom_processed_data.json datasets/custom_output.json datasets/model.pt

#     predictions = json.load(open('datasets/custom_output.json'))
#     sent_tags = []
#     annotation_list = predictions[0]['annotations'][0]['result']
#     for i,annotation in enumerate(annotation_list):
#       sent_tags.append((annotation['value']['text'],annotation['value']['labels'][0]))
#       if i < len(annotation_list)-1 and annotation_list[i+1]['value']['start'] > annotation['value']['end']:
#         sent_tags.append((cleaned_text[annotation['value']['end'] :annotation_list[i+1]['value']['start']],None))

#     return sent_tags

# ### sample input data 
# input_txt = '''                                                         REPORTABLE\n\n  IN THE SUPREME COURT OF INDIA\n CRIMINAL APPELLATE JURISDICTION\n\nCRIMINAL APPEAL NO. 1053 OF 2016\n\nNARESH KUMAR alias NITU                            ..........APPELLANT\n                                                   ......RESPONDENT\n                                           Versus\nSTATE OF HIMACHAL PRADESH\n\n                                       JUDGMENT\n NAVIN SINHA, J.\n The acquittal of the appellant by the Special Judge, Shimla in Sessions Trial No.7-S/7/2012, from the charge under Sections 20 and 61 of the Narcotic Drugs and Psychotropic Substances Act, 1985 (hereinafter referred as `the Act') has been reversed by the High Court. The appellant has been sentenced to fifteen years imprisonment and fine of Rs.2,00,000/-. Thus the present appeal.\n 2. The appellant is stated to have been apprehended at Majhotli, by the police party, on suspicion, at about 6.15 A.M.\nafter he disembarked from the State public transport bus going from Nerwa to Chamunda. Two kilograms of Charas is stated to have been recovered from a bag in his possession in presence of PW-2, Sita Ram an independent witness. DW-1 Shayam Singh, the depot in-charge at Nerwa, and DW-2 Khem Raj, the conductor of the bus in question were examined as defence witnesses.\n 3. After consideration of the entirety of the evidence, particularly that of PW-2 and DW-2, and also noticing that PW-1, Constable Rakesh Kumar, an eye-witness to the incident, had not been mentioned as a witness in the seizure memo Exhibit PW-1/B, the Special Judge opined that two theories had emerged with regard to the accusations against the appellant. The appellant was acquitted, giving him the benefit of doubt.\n 4. The High Court reversed the acquittal, holding that PW-2 had admitted his signatures on Exhibit PW-1/B, the bag along with the narcotic, Ex.PW-2/A seal impression, Ex.PW-2/D the arrest memo and the Ex.PW-2/E personal search memo. No complaint had been lodged by the witness that he had been compelled by the police to sign the documents under pressure.\nThe statements of the official witnesses, PW-1 Rakesh Kumar and PW-6 Head Constable Parmanand, were trustworthy, inspiring confidence, and could not be rejected only on the ground that they were police personnel. Any discrepancy with regard to distance and travelling time between Nerwa and Majhotli could be attributed to memory loss with passage of time, and was not required to be with mathematical precision. The time with regard to purchase of bus ticket had not been established.\n 5. Learned counsel for the appellant submits that he has been falsely implicated. He was deboarded from the bus at Chopal after it arrived from Majhotli and the narcotics planted on him. There had been no recovery from him. The bus ticket evidently reveals that he boarded the bus at 6:51 A.M. The question of his being apprehended at 6:15 A.M. simply does not arise. The road distance from Nerwa to Majhotli was 26 kms. as deposed by DW-1. In the hills, it would have taken the bus at least one hour or more, to cover this distance. The bus could not have reached Majhotli till 8:00 A.M. or thereafter. PW-2, the independent witness, deposed that he was stopped by the police at about 10.30 A.M. at Majhotli and after checking his motor cycle papers was allowed to leave. He was called to the Police Station at 1.00 P.M. and asked to put his signatures on certain documents. The witness denied any search, seizure and recovery from the appellant in his presence. If an independent witness was available, and relied upon by the prosecution, his evidence could not be discarded without reason, to hold that the police version was the truth. Additional submissions were made with regard to non-compliance with Section 50 of the Act, as there was personal search also, and that the complainant himself could not be the investigating officer. Reliance was placed on Basappa vs. State of Karnataka, 2014 (5) SCC 154 that if two views were possible, the acquittal ought not to have been interfered with in appeal.\n 6. We have considered the submissions on behalf of the parties, and also examined the evidence and other materials on record.\n 7. The public bus, on which the appellant was traveling, was going from Nerwa to Chamunda. The ticket issued to the appellant Exhibit DX, proved by the bus Conductor DW-2, bears the time of issuance 6.51 A.M., visible to the naked eye. The distance from Nerwa to Majhotli, is 26 kms. as deposed by DW-1.\nWe find substance in the submission on behalf of the appellant, that the travelling time for the bus, in the hills, for this distance would be one hour or more. Prima facie, the prosecution story that the appellant was apprehended at Majhotli at 6.15 A.M.\nbecomes seriously doubtful if not impossible. The bus would have reached Majhotli at about 8.00 A.M. or thereafter only. The conclusion of the High Court that passage of time, and memory loss, were sufficient explanation for the time difference, is held to be perverse, and without proper consideration of Exhibit DX.\nPW-2, the independent witness has stated that he was stopped at Majhotli by the police at 10.30 A.M. and was allowed to leave after verification of his motor cycle papers. The witness has specifically denied that the appellant was apprehended in his presence and that any search, seizure and recovery was conducted in his presence. He had deposed that he was called to the police station at 1:00 P.M. and asked to sign the papers. The witness was declared hostile. This aspect has not been considered by the High Court, which proceeded on the only assumption that the signatures were admitted.\n 8. In a case of sudden recovery, independent witness may not be available. But if an independent witness is available, and the prosecution initially seeks to rely upon him, it cannot suddenly discard the witness because it finds him inconvenient, and place reliance upon police witnesses only. In the stringent nature of the provisions of the Act, the reverse burden of proof, the presumption of culpability under Section 35, and the presumption against the accused under Section 54, any reliance upon Section 114 of the Evidence Act in the facts of the present case, can only be at the risk of a fair trial to the accused.\nKaramjit Singh vs. State (Delhi Administration), AIR 2003 SC 1311, is distinguishable on its facts as independent witness had refused to sign because of the fear of terrorists. Likewise S.\n Jeevananthanan vs. State, 2004(5) SCC 230, also does not appear to be a case where independent witnesses were available.\n 9. The presumption against the accused of culpability under Section 35, and under Section 54 of the Act to explain possession satisfactorily, are rebuttable. It does not dispense with the obligation of the prosecution to prove the charge beyond all reasonable doubt. The presumptive provision with reverse burden of proof, does not sanction conviction on basis of preponderance of probability. Section 35 (2) provides that a fact can be said to have been proved if it is established beyond reasonable doubt and not on preponderance of probability. That the right of the accused to a fair trial could not be whittled down under the Act was considered in Noor Aga vs. State of Punjab, (2008) 16 SCC 417, observing:- \"58......An initial burden exists upon the prosecution and only when it stands satisfied, would the legal burden shift. Even then, the standard of proof required for the accused to prove his innocence is not as high as that of the prosecution. Whereas the standard of proof required to prove the guilt of the accused on the prosecution is \"beyond all reasonable doubt\" but it is \"preponderance of probability\" on the accused. If the prosecution fails to prove the foundational facts so as to attract the rigours of Section 35 of the Act, the actus reus which is possession of contraband by the accused cannot be said to have been established.\n 59. With a view to bring within its purview the requirements of Section 54 of the Act, element of possession of the contraband was essential so as to shift the burden on the accused. The provisions being exceptions to the general rule, the generality thereof would continue to be operative, namely, the element of possession will have to be proved beyond reasonable doubt.\"\n 10. In the facts of the present case, and the nature of evidence as discussed, the prosecution had failed to establish the foundational facts beyond all reasonable doubt. The special judge committed no error in acquitting the appellant. The High Court ought not to have interfered with the same. The submissions regarding non-compliance with Section 50 of the Act, or that the complainant could not be the investigating officer are not considered necessary to deal with in the facts of the case.\n 11. In Basappa (supra), it was observed that the High Court before setting aside an order of acquittal was required to record a finding that the conclusions of the Trial Court were so perverse and wholly unreasonable, so as not to be a plausible view by misreading and incorrect appreciation of evidence. The conclusions of the High Court in the facts of the present case are more speculative, based on conjectures and surmises, contrary to the weight of the evidence on record.\n 12. The order of the High Court is set aside. The acquittal of the appellant ordered by the Special Judge is restored. The appellant is set at liberty forthwith, unless wanted in any other case. The appeal is allowed.\n ........................................J.\n                             (L. Nageswara Rao) .......................................J.\n                                     (Navin Sinha) New Delhi, July 27, 2017'''


# color_map={"PREAMBLE": "LightCyan", "FAC": "Yellow","RLC":"LightSkyBlue",
# "ISSUE":"Aquamarine","ARG_PETITIONER":"LightGreen","ARG_RESPONDENT":"Lavender","ANALYSIS":"Thistle","STA":"PaleGoldenrod","PRE_RELIED":"YellowGreen","PRE_NOT_RELIED":"Gold",
# "RATIO":"Orange","RPC":"PeachPuff","NONE":"Linen" }


# ########## launch Gradio UI
# iface = gr.Interface(
#     predict_rhetorical_roles,
#     [
#         gr.inputs.Textbox(
#             lines=20, default=input_txt)
#     ],
#     gr.outputs.HighlightedText(color_map))
# gradio_output = iface.launch()

# ########## play beep 
# from google.colab import output
# output.eval_js('new Audio("https://static.sfdict.com/audio/C07/C0702600.mp3").play()')